## Introduction
Investigating a reported intermittent problem with API 2.0
PoC Client is reporting that they are getting intermittent errors with the following:
1. Creating a savings account from the API
1. Changing the status to approved from API
1. Intermittently step 2 is failing

GitHub: https://github.com/MkershMambu/MambuAPINotebook/blob/master/IntermittentProblemInvestigating.ipynb

Viewer: https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/IntermittentProblemInvestigating.ipynb

In [6]:
%run API.py
%run ENV.py
from IPython.core.display import HTML
setENV(ENV)

In [2]:
%%writefile JSONBODY
{
  "overdraftSettings": {
	"allowOverdraft": true,
	"overdraftLimit": 0
  },
  "accountType": "CURRENT_ACCOUNT",
  "name": "MKCurTest1",
  "accountHolderKey": "8a8186ac692678910169288509c606bc",
  "productTypeKey": "8a8186aa69491c25016949888aeb10d2",
  "currencyCode": "EUR",
  "accountHolderType": "CLIENT"
}

Overwriting JSONBODY


In [3]:
%%writefile JSONBODY2
{
  "action": "APPROVE",
  "notes": "Approved from the API"
}

Overwriting JSONBODY2


In [4]:
def testScenario(deleteIt=True):
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    # Step 1 -------------------- Create the account
    r = POST('{{env1}}/deposits', headers=HEADERS, params=PARAMS, body="JSONBODY")
    encodedKey = r.json()['encodedKey']
    if r.status_code != 201:
        print("Failed to create account")
    else:
        print("Created new account - {0}".format(encodedKey))

    # Step 2 -------------------- Change the Status to approved
    url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
    r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY2")
    if r.status_code != 200:
        print("Failed to change Status of account to approved")
    else:
        print("Changed Status to approved - successful")

    if deleteIt == True:
        # Step 3 -------------------- Delete the account
        url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
        r = DELETE(url, headers=HEADERS, params=PARAMS)
        if r.status_code != 204:
            print("Failed to delete account")
        else:
            print("Delete account - successful")

In [21]:
for i in range(5):
    print("[{0}]".format(i), end='')
    testScenario(True)

[0]Created new account - 8a8187786a21e99b016a21fab5770079
Changed Status to approved - successful
Delete account - successful
[1]Created new account - 8a8187da6a2183a3016a21faaccc01f6
Changed Status to approved - successful
Delete account - successful
[2]Created new account - 8a8187d36a210122016a21fab9ab3bbc
Changed Status to approved - successful
Delete account - successful
[3]Created new account - 8a8187786a21e99b016a21fab577007c
Changed Status to approved - successful
Delete account - successful
[4]Created new account - 8a8187d36a210122016a21fb18613bbf
Changed Status to approved - successful
Delete account - successful


### Trying an alternative version of the scenario - testScenario2
This should be faster between API operations because there is no reading of files

This is also failing to duplicate the issue though??

In [18]:
jsonData1 = readFile("JSONBODY")
jsonData2 = readFile("JSONBODY2")

In [19]:
def testScenario2(deleteIt=True):
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    # Step 1 -------------------- Create the account
    r = POST2('{{env1}}/deposits', headers=HEADERS, params=PARAMS, data=jsonData1)
    encodedKey = r.json()['encodedKey']
    if r.status_code != 201:
        print("Failed to create account")
    else:
        print("Created new account - {0}".format(encodedKey))

    # Step 2 -------------------- Change the Status to approved
    url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
    r = POST2(url, headers=HEADERS, params=PARAMS, data=jsonData2)
    if r.status_code != 200:
        print("Failed to change Status of account to approved")
    else:
        print("Changed Status to approved - successful")

    if deleteIt == True:
        # Step 3 -------------------- Delete the account
        url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
        r = DELETE(url, headers=HEADERS, params=PARAMS)
        if r.status_code != 204:
            print("Failed to delete account")
        else:
            print("Delete account - successful")

In [20]:
for i in range(5):
    print("[{0}]".format(i), end='')
    testScenario2(True)

[0]Created new account - 8a8186c16a21796b016a21f7650f05e3
Changed Status to approved - successful
Delete account - successful
[1]Created new account - 8a8187da6a2183a3016a21f77fd801f2
Changed Status to approved - successful
Delete account - successful
[2]Created new account - 8a8187786a21e99b016a21f78b230076
Changed Status to approved - successful
Delete account - successful
[3]Created new account - 8a8186c16a21796b016a21faaac105e7
Changed Status to approved - successful
Delete account - successful
[4]Created new account - 8a8186696a21009e016a21f778513bff
Changed Status to approved - successful
Delete account - successful


In [14]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = DELETE(url, headers=HEADERS, params=PARAMS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
204
Now the JSON:



In [13]:
encodedKey = "8a8186696a21009e016a21f02a363bbe"

In [54]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
# Parameters below set up pagination and only returns max 20
PARAMS = {'detailsLevel': 'FULL'}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = GET(url, headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 404
JSON Response:
{
    "errors": [
        {
            "errorCode": 428,
            "errorReason": "INVALID_DEPOSIT_ACCOUNT_ID"
        }
    ]
}


In [25]:
%%writefile JSONBODY3
{
  "action": "UNDO_APPROVE",
  "notes": "Approved from the API"
}

Writing JSONBODY3


In [26]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY3")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
200
Now the JSON:
{"encodedKey":"8a8186c16a21796b016a21d0cce0055f","creationDate":"2019-04-15T18:24:53+02:00","lastModifiedDate":"2019-04-15T18:29:44+02:00","id":"MGQN492","name":"MKCurTest1","accountHolderType":"CLIENT","accountHolderKey":"8a8186ac692678910169288509c606bc","accountState":"PENDING_APPROVAL","productTypeKey":"8a8186aa69491c25016949888aeb10d2","accountType":"CURRENT_ACCOUNT","currencyCode":"EUR","assignedBranchKey":"8a8186ac692678910169287cf43606af","internalControls":{},"overdraftSettings":{"allowOverdraft":true,"overdraftLimit":0E-10},"interestSettings":{"interestPaymentSettings":{"interestPaymentDates":[]}},"overdraftInterestSettings":{"interestRateSettings":{"encodedKey":"8a8186c16a21796b016a21d0cce00560","interestRate":0E-20,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED","interestRateSource":"FIXED_INTEREST_RATE"}},"balances":{"overdraftAmount":0E-10,"technicalOverdraftAmount":0E